### Задание  
Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации  
2. Рассмотреть 2-а варианта сеточек  
    2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/  
    2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
import numpy as np
import pandas as pd

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

### Чтение датасета

In [2]:
data = pd.read_excel("отзывы за лето.xls")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


### Предобработка

https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb
https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_udpipe.py

In [3]:
def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken


def clean_token(token, misc):
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma

In [4]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [5]:
from ufal.udpipe import Model, Pipeline
import wget
import os
import sys

udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    wget.download(udpipe_model_url)

print('\nLoading the model...', file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')


Loading the model...


In [6]:
def preprocess_text(txt, keep_pos):
    txt = str(txt)
    output = process(process_pipeline, text=txt, keep_pos=keep_pos)
    return ' '.join(output)

In [7]:
preprocess_text(data['Content'].values[1], keep_pos=True)

'в_ADP целое_NOUN удобноный_ADJ приложение_VERB из_ADP минус_NOUN хотеть_VERB слишком_ADV большой_ADJ доступ_NOUN к_ADP персональный_ADJ данные_NOUN в_ADP телефон_NOUN приходиться_VERB пользоваться_VERB в_ADP ограничивать_VERB режим_NOUN'

In [8]:
data['text'] = data['Content'].apply(preprocess_text, keep_pos=False)
data['text_with_tag'] = data['Content'].apply(preprocess_text, keep_pos=True)

data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [9]:
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,text_with_tag,target
0,5,It just works!,2017-08-14,it just works,it_PROPN just_X works_X,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноный приложение из минус хотеть с...,в_ADP целое_NOUN удобноный_ADJ приложение_VERB...,1
2,5,Отлично все,2017-08-14,отличный все,отличный_ADJ все_PRON,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на x % работа антивирус далеко ...,стать_VERB зависать_VERB на_ADP x_NUM %_SYM ра...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобный работать быстро,очень_ADV удобный_ADV работать_VERB быстро_ADV,1


### Разбиваем на train и test 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data['text_with_tag'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [11]:
X_train

547                                           классный_ADV
1863     невозможный_ADJ использовать_VERB на_ADP рутов...
13949      работать_VERB хорошо_ADV без_ADP нарекание_NOUN
11501                                         отличный_ADJ
6238     я_PRON не_PART мочь_VERB понимать_VERB почему_...
                               ...                        
5897                           спсиба_NOUN за_ADP все_PRON
19812                                           супер_NOUN
14734                                          удобный_ADJ
4851                                            супер_NOUN
19238                                 все_PRON хороший_ADJ
Name: text_with_tag, Length: 15798, dtype: object

### Векторизуем

In [12]:
vectorizer = TextVectorization(standardize=None, max_tokens=20000, output_sequence_length=200)
vectorizer.adapt(data['text_with_tag'].values)

In [13]:
x_train = vectorizer(np.array([[s] for s in X_train.values])).numpy()
x_test = vectorizer(np.array([[s] for s in X_test.values])).numpy()

In [14]:
x_train.shape

(15798, 200)

In [15]:
num_classes = 2

In [16]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [17]:
y_train.shape

(15798, 2)

### Использование pre-trained word embeddings

https://keras.io/examples/nlp/pretrained_word_embeddings/

Используем данные модели http://vectors.nlpl.eu/repository/20/220.zip

In [18]:
path_to_glove_file = 'model.txt'
embeddings_index = {}
with open(path_to_glove_file, encoding='utf8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 249334 word vectors.


In [19]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [20]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0
misses_words = []

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        misses_words.append(word)

print("Converted %d words (%d misses)" % (hits, misses))

Converted 5832 words (6110 misses)


In [21]:
misses_words

['',
 '[UNK]',
 'не_PART',
 'и_CCONJ',
 'все_PRON',
 'в_ADP',
 'я_PRON',
 'на_ADP',
 'с_ADP',
 'что_SCONJ',
 'по_ADP',
 'но_CCONJ',
 'после_ADP',
 'а_CCONJ',
 'что_PRON',
 'это_PRON',
 'x_NUM',
 'весь_DET',
 'за_ADP',
 'весь_PRON',
 'быть_AUX',
 'при_ADP',
 'и_PART',
 'только_PART',
 'вы_PRON',
 'у_ADP',
 'просто_PART',
 'как_SCONJ',
 'этот_DET',
 'он_PRON',
 'для_ADP',
 'свой_DET',
 'без_ADP',
 'из_ADP',
 'то_PRON',
 'бы_PART',
 'все_PART',
 'или_CCONJ',
 'мой_DET',
 'через_ADP',
 'до_ADP',
 'если_SCONJ',
 'к_ADP',
 'ваш_DET',
 'который_PRON',
 'такой_DET',
 'они_PRON',
 'даже_PART',
 'когда_SCONJ',
 'какой_DET',
 'же_PART',
 'то_SCONJ',
 'о_ADP',
 'от_ADP',
 'ничто_PRON',
 'нормальный_NOUN',
 'все_DET',
 'пока_SCONJ',
 'чтобы_SCONJ',
 'ирать_VERB',
 'хотя_SCONJ',
 'под_ADP',
 'оно_PRON',
 'из-за_ADP',
 'ни_CCONJ',
 'ни_PART',
 'кто_PRON',
 'никакой_DET',
 'тот_DET',
 'она_PRON',
 'вот_PART',
 'ну_PART',
 'супер_VERB',
 'ирать_NOUN',
 'чем_SCONJ',
 'meizu_X',
 'себя_PRON',
 'между_ADP

In [22]:
embedding_matrix.shape

(11944, 300)

In [23]:
def get_model(embedding_layer):
    model = Sequential()
    model.add(embedding_layer)
    model.add(Conv1D(128, 3))
    model.add(Activation("relu"))
    model.add(GlobalMaxPool1D())
    model.add(Dense(10))
    model.add(Activation("relu"))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

#### Вариант без дообучения параметров слоя Embedding

In [24]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)
model = get_model(embedding_layer)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 300)         3583200   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         115328    
_________________________________________________________________
activation (Activation)      (None, None, 128)         0         
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [25]:
epochs = 20
batch_size = 512
print_batch_n = 100

In [26]:
tensorboard=TensorBoard(log_dir='./logs1', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
 1/28 [>.............................] - ETA: 0s - loss: 0.8578 - accuracy: 0.6777WARNING:tensorflow:From C:\Users\Kartsev.ES\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
28/28 [==============================] - 9s 337ms/step - loss: 0.3263 - accuracy: 0.8721 - val_loss: 0.2534 - val_accuracy: 0.9063
Epoch 2/20
28/28 [==============================] - 9s 326ms/step - loss: 0.2013 - accuracy: 0.9249 - val_loss: 0.2357 - val_accuracy: 0.9076
Epoch 3/20
28/28 [==============================] - 9s 333ms/step - loss: 0.1558 - accuracy: 0.9472 - val_loss: 0.2274 - val_accuracy: 0.9025
Epoch 4/20
28/28 [==============================] - 9s 329ms/step - loss: 0.1247 - accuracy: 0.9629 - val_loss: 0.2221 - val_accuracy: 0.9070
Epoch 5/20
28/28 [==============================] - 9s 331ms/st

In [27]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 90ms/step - loss: 0.2196 - accuracy: 0.9124


Test loss: 0.21956004202365875
Test accuracy: 0.9124050736427307


#### Вариант с дообучением параметров слоя Embedding

In [28]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=True,
)
model = get_model(embedding_layer)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         3583200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         115328    
_________________________________________________________________
activation_3 (Activation)    (None, None, 128)         0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [29]:
tensorboard=TensorBoard(log_dir='./logs2', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 15s 532ms/step - loss: 0.4445 - accuracy: 0.8344 - val_loss: 0.2689 - val_accuracy: 0.8949
Epoch 2/20
28/28 [==============================] - 15s 523ms/step - loss: 0.2294 - accuracy: 0.9112 - val_loss: 0.2330 - val_accuracy: 0.9063
Epoch 3/20
28/28 [==============================] - 14s 508ms/step - loss: 0.1815 - accuracy: 0.9353 - val_loss: 0.2140 - val_accuracy: 0.9063
Epoch 4/20
28/28 [==============================] - 14s 511ms/step - loss: 0.1446 - accuracy: 0.9511 - val_loss: 0.1962 - val_accuracy: 0.9139
Epoch 5/20
28/28 [==============================] - 14s 514ms/step - loss: 0.1145 - accuracy: 0.9642 - val_loss: 0.1944 - val_accuracy: 0.9146
Epoch 6/20
28/28 [==============================] - 14s 513ms/step - loss: 0.0904 - accuracy: 0.9746 - val_loss: 0.1938 - val_accuracy: 0.9171
Epoch 7/20
28/28 [==============================] - 14s 517ms/step - loss: 0.0708 - accuracy: 0.9826 - val_loss: 0.2016 - val_accuracy: 0.9158

In [30]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 93ms/step - loss: 0.1908 - accuracy: 0.9258


Test loss: 0.19077487289905548
Test accuracy: 0.9258227944374084


### Случайная инициализация весов слоя Embedding

In [31]:
model = get_model(Embedding(num_tokens, embedding_dim))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 300)         3583200   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         115328    
_________________________________________________________________
activation_6 (Activation)    (None, None, 128)         0         
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                

In [32]:
tensorboard=TensorBoard(log_dir='./logs3', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 15s 536ms/step - loss: 0.4732 - accuracy: 0.8197 - val_loss: 0.3369 - val_accuracy: 0.8430
Epoch 2/20
28/28 [==============================] - 15s 519ms/step - loss: 0.2613 - accuracy: 0.8473 - val_loss: 0.2186 - val_accuracy: 0.8449
Epoch 3/20
28/28 [==============================] - 15s 540ms/step - loss: 0.1806 - accuracy: 0.8902 - val_loss: 0.1717 - val_accuracy: 0.9203
Epoch 4/20
28/28 [==============================] - 15s 520ms/step - loss: 0.1289 - accuracy: 0.9504 - val_loss: 0.1658 - val_accuracy: 0.9278
Epoch 5/20
28/28 [==============================] - 14s 513ms/step - loss: 0.0975 - accuracy: 0.9648 - val_loss: 0.1685 - val_accuracy: 0.9253


In [33]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 91ms/step - loss: 0.1644 - accuracy: 0.9327


Test loss: 0.16439098119735718
Test accuracy: 0.9326582551002502


__ВЫВОДЫ:__ Вариант с использованием pre-trained word embeddings выглядит хуже, даже в режиме дообучения параметров слоя. Возможно это связано с особенностями исходных текстов. Эмбединги для более половины слов словаря не были сопоставлены из данных выбранной модели. В словах исходных текстов, для которых не удалось получить эмбединг, есть грамматические ошибки, сокращения, техническая терминология, тех. сленг и т.п. Возможно стоило выбрать данные другой модели.